In [1]:
from mlx_lm import load, generate
import pandas as pd

In [2]:
# %%bash
# python -m mlx_lm.convert --hf-path jphme/em_german_leo_mistral --mlx-path model/ -q 

In [3]:
# TODO: Choose one
# df = pd.read_csv("test.csv")
df = pd.read_csv("interpreted_data_mlx_test2.csv")

In [4]:
# TODO: Choose which row to start from
n=12213
inter_df = df.iloc[n:]

In [5]:
inter_df.head()

,Unnamed: 0,patientid,sex,age,ANA,EXA,DIA_text,DIA_code,OP_text,OP_code,EXA_formatted,EXA_interpreted
12213,12213,62388.0,male,43.0,Vorstellung präop; trägt Brace täglich; keine ...,Beinachse: Gangbild: an...,Sonstige Spontanruptur eines oder mehrerer Bän...,M23.61 Z,NaN,NaN,beinachse gangbild an u...,NaN
12214,12214,62391.0,male,63.0,NaN,NaN,Z.n.AC re Schulter m.RM-Debridement,M23.3- Z,NaN,NaN,NaN,NaN
12215,12215,62395.0,female,19.0,Pat stellt sich mit den MRT Bildern vor.,li Knie: keine Rötung/Überwärmung; klin.kein E...,Habituelle Luxation der Patella,M22.0 G,NaN,NaN,"links knie keine rötung/überwärmung, kein ergu...",NaN
12216,12216,62399.0,male,28.0,Beruf; Tunnelelektriker; Sport: Capoira; Schwi...,Beinachse: genuvarum 2 QF Gangbild: flüssig l...,Chondropathia patellae links,M22.4 G,NaN,NaN,beinachse genuvarum 2 qf gangbild flüssig lin...,NaN
12217,12217,62400.0,male,57.0,6 Wochen nach OP. Pat. berichtet nach 4 Tagen ...,Beinachse: genuvarum 3 QF; Gangbild: flüssig l...,IV° Chondromalazie mediale Femurkondyle li.Knie,M22.4 Z,NaN,NaN,gangbild flüssig links knie keine überstreckun...,NaN


In [6]:
model, tokenizer = load("model/")

In [7]:
few_shot_examples = [
    ("links knie diskr schwellung, kein anteromedialer druckschmerz med äußerechts Seite des Kniegelenks, kein druckschmerz kniescheiben, keine meniskuszeichen rechts knie keine schwellung, druckschmerz proximal seitlich kniescheibe", 
     "Mein linkes Knie hat keine Schwellung, und es gibt keine Schmerzen beim Drücken auf der äußerechts Seite des Kniegelenks und der Kniescheibe. Mein rechtes Knie hat auch keine Schwellung, aber in der Nähe der Seite der Kniescheibe, gibt es Schmerzen beim Drücken."),
    ("rechts knie deutlicher schwellung, keine überstreckung des knies, streckdefizit des knies, angemessene beugung des knies, keine rötung", 
     "Mein rechtes Knie hat eine Schwellung, ich kann es nicht überstrecken und auch nicht komplett strecken. Die Beugung des Knies ist nur angemessen möglich. Mein Knie ist nicht gerötet."),
]

# Construct the few-shot prompt
instruction = "Du bist ein Patient mit den folgenden Symptomen. Ersetze alle lateinischen und deutschen Fachbegriffe und formuliere es umgangssprachlich aus deiner Sicht."
few_shot_prompt = f"{instruction}\n\n" + "\n\n".join([f"USER: {inp}\nASSISTANT: {out}" for inp, out in few_shot_examples])

In [8]:
def generate_response(text):
    if (text != "") and (text is not None) and not (isinstance(text, float)):
        prompt = f"{few_shot_prompt}\n\nUSER: {text}\nASSISTANT:"
        generated_text = generate(model, tokenizer, prompt=prompt, max_tokens=200, verbose=False)
        return generated_text
    return ""

In [9]:
def remove_after_newline(text):
    return text.split('\n')[0] if isinstance(text, str) else text

In [10]:
for idx, row in inter_df.iterrows():
    text = row['EXA_formatted']
    ouput_text = generate_response(text)
    inter_df.at[idx, 'EXA_interpreted'] = ouput_text

In [11]:
inter_df["EXA_interpreted"] = inter_df["EXA_interpreted"].apply(remove_after_newline)

/var/folders/sg/lvbmg7vj0wsc8c5sn3vhh4rc0000gn/T/ipykernel_4491/1584057196.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  inter_df["EXA_interpreted"] = inter_df["EXA_interpreted"].apply(remove_after_newline)


In [12]:
inter_df.to_csv("interpreted_data_mlx_test2_2.csv", index=False)

In [13]:
# TODO: Use this value when rerunning notebook!
n = inter_df.EXA_interpreted.count()

In [14]:
n

7917